# Filtrado colaborativo (preprocesamiento)

**Autor**: Arturo Sánchez Palacio

Basado en: https://github.com/lazyprogrammer

**Fecha de última revisión: 15/I/2020**

Estos son los módulos que usaremos en esta tarea:

__Nota.__ Si no se dispone de alguno de los módulos se puede instalar desde Jupyter mediante la instrucción:

dependiendo de si se emplea pip o Anaconda. El módulo debe ser instalado antes de ser importado.

In [1]:
import pandas as pd
import pickle

#### To do. Cargamos los datos a partir del archivo `rating.csv`:

In [4]:
rating_df =  pd.read_csv("data/rating.csv")

#### To do. Comprobamos el tamaño del archivo y que se haya leído correctamente:

In [14]:
df = rating_df
print('Size of the table: ' + str(df.shape))

Size of the table: (20000263, 4)


In [15]:
df.describe()

,userId,movieId,rating
count,2.000026e+07,2.000026e+07,2.000026e+07
mean,6.904487e+04,9.041567e+03,3.525529e+00
std,4.003863e+04,1.978948e+04,1.051989e+00
min,0.000000e+00,1.000000e+00,5.000000e-01
25%,3.439400e+04,9.020000e+02,3.000000e+00
50%,6.914000e+04,2.167000e+03,3.500000e+00
75%,1.036360e+05,4.770000e+03,4.000000e+00
max,1.384920e+05,1.312620e+05,5.000000e+00


In [16]:
df.tail()

,userId,movieId,rating,timestamp
20000258,138492,68954,4.5,2009-11-13 15:42:00
20000259,138492,69526,4.5,2009-12-03 18:31:48
20000260,138492,69644,3.0,2009-12-07 18:10:57
20000261,138492,70286,5.0,2009-11-13 15:42:24
20000262,138492,71619,2.5,2009-10-17 20:25:36


Parece que se ha cargado correctamente.

Nuestra primera tarea es construir la matriz de usuarios-valoraciones.

Cuando trabajamos con datos y especialmente con grandes cantidades de datos siempre debemos tener en mente la eficiencia computacional. Así a la hora de construir la matriz nos interesa que los índices estén aprovechados, es decir:

In [8]:
print(df.userId.min(), df.userId.max())

1 138493


Que los usuarios estén identificados por índices consecutivos sin huecos. Estos huecos supondrían filas vacías en nuestra matriz que ocuparían memoria sin aportar ninguna información ya que el índice es simplemente informativo.

#### To do. Comprobar cuántos índices distintos de usuario hay almacenados. ¿Hay huecos en el índice?

In [20]:
#Check unic values
print("Number of unique values of users Id: " + str(df['userId'].nunique()))

Number of unique values of users Id: 138493


In [21]:
#Check null fields
print("Number of null values of users Id: " + str(df['userId'].isnull().sum()))

Number of null values of users Id: 0


Como la indexación en Python comienza en 0 restamos uno a los índices:

In [11]:
df.userId = df.userId - 1

In [12]:
print(df.userId.min(), df.userId.max())

0 138492


Realizamos la misma comprobación para los índices asociados a las películas:

In [13]:
print(df.movieId.min(), df.movieId.max())

1 131262


In [18]:
#mismo que apartado anterior
print('Nº of unic vlaues: ' + str(df['movieId'].nunique()))
print('Nº of null fields: ' +str(df['movieId'].isnull().sum()))

Nº of unic vlaues: 26744
Nº of null fields: 0


En este caso los índices no están asignados secuencialmente (puede que se hayan descartado películas sin votos, por ejemplo). Realizamos una reindexación de las películas para tenerlas identificadas de manera secuencial:

#### To do. Reindexar las películas para tenerlas identificadas de manera secuencial:

In [24]:
unique_movie_ids = set(df.movieId.values) # In a set type there are not repeted values
diccionario_indice_peli = {} #key: film, value: new index

nuevo_indice = 0

for pelicula in unique_movie_ids:
    diccionario_indice_peli[pelicula] = nuevo_indice
    nuevo_indice += 1
    
df['movie_idx'] = df.apply(lambda row: diccionario_indice_peli[row.movieId], axis=1)


__Nota.__ Estamos añadiendo 20 millones de datos así que es normal que tarde un rato.

Comprobamos que la columna se ha añadido al dataframe y que todo es correcto:

In [25]:
df.head()

,userId,movieId,rating,timestamp,movie_idx
0,0,2,3.5,2005-04-02 23:53:47,2
1,0,29,3.5,2005-04-02 23:31:16,29
2,0,32,3.5,2005-04-02 23:33:39,32
3,0,47,3.5,2005-04-02 23:32:07,47
4,0,50,3.5,2005-04-02 23:29:40,50


In [26]:
print(df.movie_idx.min(), df.movie_idx.max())

0 26743


Finalmente en este ejercicio no usaremos la fecha así que podemos eliminar la columna para ahorrar memoria:

#### To do. Eliminar la columna con las fechas:

In [27]:
df.drop(columns="timestamp")

,userId,movieId,rating,movie_idx
0,0,2,3.5,2
1,0,29,3.5,29
2,0,32,3.5,32
3,0,47,3.5,47
4,0,50,3.5,50
5,0,112,3.5,112
6,0,151,4.0,151
7,0,223,4.0,222
8,0,253,4.0,252
9,0,260,4.0,259


In [ ]:
df.head()

Tras realizar todas las transformaciones guardamos los datos en un nuevo archivo para no perder tiempo en realizar los cambios en un futuro:

#### To do. Guardar los datos en un archivo csv llamado `preprocessed_rating.csv`:

Hasta aquí nuestro primer preprocesamiento de datos.

Este método no es escalable. Si se tienen conocimientos de Spark la implementación se podría hacer de manera paralelizada pero como dichos conocimientos no son requisitos para este curso lo que haremos será reducir nuestros datos seleccionando un subconjuntos de usuarios y películas. Como discutimos previamente en la teoría la idea más interesante es elegir las películas que más veces han sido votadas y los usuarios que más veces han votado.

Como ahora nuestros índices son secuenciales podemos afirmar que:

#### To do. Calcular el número de películas y el número de usuarios:

La función `Counter` del módulo `collections` nos permite hacer un conteo eficiente de valores agrupados devolviéndonos un objeto counter (similar a un diccionario con clave el índice y valor el número de apariciones). 

__Nota.__ El módulo `collections` viene automáticamente instalado.

In [ ]:
from collections import Counter

Ejemplo:

In [ ]:
Counter(df.movie_idx)[10] #veces que la película 10 ha sido votada

#### To do. Contamos pues los votos emitidos por cada usuario y los votos sobre cada película respectivamente:

Ahora fijamos el tamaño de muestra que vamos a elegir. En un proyecto real estos valores no son dados ni elegidos al azar. Se realizan diferentes pruebas y se comparan los resultados. En esta primera aproximación simplemente elegiremos valores que permitan un fácil manipulación de los datos:

In [ ]:
m =  # número de películas elegidas
n =  # número de usuarios elegidos

`Counter` incluye el método `most_common()` que nos permite extraer los índices asociados a los x valores más frecuentes:

In [ ]:
user_ids = [u for u, c in user_ids_count.most_common(n)] # para cada índice y valor en el counter extraigo el índice
movie_ids = [m for m, c in movie_ids_count.most_common(m)]

Una vez elegidos los índices de las películas y usuarios con los que trabajaremos extraemos todos los votos de dichos usuarios sobre dichas películas de todos los datos iniciales, rechazando todas aquellas valoraciones realizadas por usuarios no seleccionados o sobre películas no seleccionadas:

#### To do. Extraer los votos de dichos usuarios sobre dichas películas rechanzando valoraciones de usuarios no seleccionadas o sobre películas no seleccionadas:

__Nota.__ El método `isin()` simplemente indica si un elemento está dentro de otro devolviendo un booleano.

__PROBLEMA.__ Ahora que hemos extraído ciertos usuarios/película y rechazado otros los índices vuelven a no ser secuenciales. Aplicamos de nuevo el mismo proceso

__Nota.__ Obviamente ahora nos damos cuenta de que realizar esto al principio era innecesario pero este ejemplo es didáctico y busca en parte reproducir las vicisitudes al trabajar con datos.

Creamos los diccionarios:

In [ ]:
nuevo_user_id_map = {}
i = 0
for antiguo in user_ids:
    nuevo_user_id_map[antiguo] = i
    i += 1
    
nuevo_movie_id_map = {}
j = 0
for antiguo in movie_ids:
    nuevo_movie_id_map[antiguo] = j
    j += 1

Aplicamos la transformación a las columnas adecuadas:

In [ ]:
df_small.loc[:, 'userId'] = df_small.apply(lambda row: nuevo_user_id_map[row.userId], axis=1)
df_small.loc[:, 'movie_idx'] = df_small.apply(lambda row: nuevo_movie_id_map[row.movie_idx], axis=1)

__Nota.__ De nuevo esto llevará un tiempo.

Una vez disponemos de los datos adecuados comprobamos que la tabla se ha guardado de forma correcta y de nuevo almacenamos los datos en un csv:

In [ ]:
df_small.head()

In [ ]:
df_small.to_csv('./data/reduced_rating.csv', index=False)

Estos son los datos finales con los que trabajaremos, sin embargo, no es su estructura óptima. Pandas es una estructura muy eficiente pero pesada en ciertos sentidos. En este caso nos interesa poder almacenar los datos en __diccionarios__.

Iterar sobre la matriz tendría coste O(NM) mientras que iterar sobre diccionarios tiene un coste O(|$\Omega$|) con |$\Omega$| el número total de valoraciones. 

Procedemos pues a almacenar la información en diccionarios:

In [ ]:
M = m #los valores son los elegidos manualmente antes
N = n

#### To do. Dividimos los datos en entrenamiento reducidos (80%) y test (20%):

In [ ]:
from sklearn.utils import shuffle

Vamos a construir tres diccionarios:

* `usuario_pelicula`: indica qué usuarios han valorado qué películas.
* `pelicula_usuario`: indica qué películas han sido valoradas por qué usuarios.
* `usuariopeli_rating`: indica dada una tupla de usuario y película como clave cual ha sido su valoración.

Inicializamos los diccionarios vacíos:

In [ ]:
usuario_pelicula = {}
pelicula_usuario = {}
usuariopeli_rating = {}

Vamos a definir una función para construir nuestros diccionarios de entrenamiento:

In [ ]:
count = 0
def crear_usuario_pelicula_y_pelicula_usuario(row):
    global count
    count += 1
    if count % 100000 == 0:
        print("Procesado: %.3f" % (float(count)/cutoff)) #pequeño truco para ver a que velocidad se procesan los datos

    i = int(row.userId) # i es el usuario
    j = int(row.movie_idx) #j es la película
    
    #Diccionario usuario_pelicula
    if i not in usuario_pelicula:
        usuario_pelicula[i] = [j] # si el usuario i no está en el diccionario lo añadimos con valor la película j
    else:
        usuario_pelicula[i].append(j) # si i ya está en el diccionario le añadimos al valor la nueva película
        
    # Diccionario pelicula_usuario
    if j not in pelicula_usuario:
        pelicula_usuario[j] = [i] # si la peli j no está en el diccionario la añadimos con valor el usuario i
    else:
        pelicula_usuario[j].append(i) # si j ya está en el diccionario el añadimos al valor el nuevo ususario
    
    # Diccionario usuariopeli_rating
    usuariopeli_rating[(i,j)] = row.rating # añado la tupla (usuario, pelicula) como clave y la valoración como valor

#### To do. Aplicar la función a los datos de entrenamiento:

Para validación solo necesitaremos un diccionario de `usuariopeli_rating` así que definimos una función que construya únicamente dicho diccionario:

#### To do. Definir una función que cree solo el diccionario `usuariopeli_rating_test`:

#### To do. Aplicar la función:

Ya hemos construido nuestros cuatro diccionarios. Finalmente los almacenamos en sendos archivos .json:

__Nota.__ `pickle` puede guardar cualquier tipo de objeto de Python.
Técnicamente son archivos binarios y no json porque la clave es un string y en los json es un entero. `pickle` es parte de Python y no requiere instalación:

In [ ]:
import pickle
with open('./data/usuario_pelicula.json', 'wb') as f:
    pickle.dump(usuario_pelicula, f)

with open('./data/pelicula_usuario.json', 'wb') as f:
    pickle.dump(pelicula_usuario, f)

with open('./data/usuariopeli_rating.json', 'wb') as f:
    pickle.dump(usuariopeli_rating, f)

with open('./data/usuariopeli_rating_test.json', 'wb') as f:
    pickle.dump(usuariopeli_rating_test, f)

Hasta aquí las labores de preprocesado realizadas. En resumen lo que hemos hecho es:

* Cargar los datos.
* Tomar una muestra de las películas más votadas y los usuarios más activos.
* Construir cuatro diccionarios, tres para entrenamiento y uno para test.